In [ ]:
import pandas as pd 
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from scipy import sparse 
from sklearn.model_selection import train_test_split
import random

class MF(object):
    def __init__(self, Y, n_factors = 2, X = None, W = None, lamda = 0.1, lr = 2, n_epochs = 50, user_based = 1, 
                 limit = 10):
#         self.f = open('danhgiaMF.dat', 'a+')
        self.Y = Y
        self.lamda = lamda
        self.n_factors = n_factors
        self.lr = lr
        self.n_epochs = n_epochs
        self.user_based = user_based
        self.limit = limit
        self.users_count = int(np.max(self.Y[:, 0])) + 1
        self.items_count = int(np.max(self.Y[:, 1])) + 1
        self.ratings_count = Y.shape[0]
        if X == None:
            self.X = np.random.randn(self.items_count, n_factors)
        if W == None:
            self.W = np.random.randn(n_factors, self.users_count)
        self.Ybar = self.Y.copy()
        
        self.bi = np.random.randn(self.items_count)
        self.bu = np.random.randn(self.users_count)
        self.n_ratings = self.Y.shape[0]
        
    def normalizeY(self):
        if self.user_based:
            user_col = 0
            item_col = 1
            n_objects = self.users_count
        else:
            user_col = 1
            item_col = 0 
            n_objects = self.items_count
        users = self.Y[:, user_col]
        self.mu = np.zeros((n_objects,))
        for i in range(n_objects):
            ids = np.where(users == i)[0].astype(int)
            ratings = self.Y[ids, 2]
            m = np.mean(ratings)
            if np.isnan(m):
                m = 0
            self.mu[i] = m
            self.Ybar[ids, 2] = ratings - self.mu[i]
#         print(self.Ybar)
    
    def get_user_rated_item(self, i):
        ids = np.where(i == self.Ybar[:, 1])[0].astype(int)
        users = self.Ybar[ids, 0].astype(int)
        ratings = self.Ybar[ids, 2]
        
        return (users, ratings)
        

    def get_item_rated_by_user(self, u):
        ids = np.where(u == self.Ybar[:, 0])[0].astype(int)
        items = self.Ybar[ids, 1].astype(int)
        ratings = self.Ybar[ids, 2]
        
        return (items, ratings)
    
    def updateXW(self):
        for i in range(self.items_count):
            users, ratings = self.get_user_rated_item(i)
            Wi = self.W[:, users]
            b = self.bu[users]
            Xi = self.X[i, :]
            
            error = Xi.dot(Wi) + self.bi[i] + b - ratings 
            grad_x = error.dot(Wi.T)/self.n_ratings + self.lamda*Xi
            grad_b = np.sum(error)/self.n_ratings + self.lamda*self.bi[i]
            self.X[i, :] -= self.lr*grad_x.reshape((self.n_factors,))
            self.bi[i]    -= self.lr*grad_b

        for u in range(self.users_count):
            items, ratings = self.get_item_rated_by_user(u)
            Xu = self.X[items, :]
            b = self.bi[items]
            Wu = self.W[:, u]
            
            error = Xu.dot(Wu) + b + self.bu[u] - ratings
            grad_w = Xu.T.dot(error)/self.n_ratings + self.lamda*Wu
            grad_b = np.sum(error)/self.n_ratings + self.lamda*self.bu[u]
            self.W[:, u] -= self.lr*grad_w.reshape((self.n_factors,))
            self.bu[u]    -= self.lr*grad_b
        
    def fit(self, x, data_size, Data_test, test_size = 0):
        self.normalizeY()
        for i in range(self.n_epochs):
            self.updateXW()
            if (i + 1) % x == 0:
                print(i + 1)
                self.RMSE(Data_test, test_size = 0)
                self.evaluate(data_size, Data_test, test_size = 0)
            
    def pred(self, u, i):
        u = int(u)
        i = int(i)
        if self.user_based:
            bias = self.mu[u]
        else: 
            bias = self.mu[i]
        pred = self.X[i, :].dot(self.W[:, u]) + self.bi[i] + self.bu[u] + bias
        
        if pred < 1:
            return 1 
        if pred > 5: 
            return 5 
        return max(0, min(5, pred))
    
    def recommend(self, u):
        ids = np.where(self.Y[:, 0] == u)[0].astype(int)
        items_rated_by_user = self.Y[ids, 1].tolist()
        a = np.zeros((self.items_count,))
        recommended_items = []
        pred = self.X.dot(self.W[:, u])
        for i in range(self.items_count):
            if i not in items_rated_by_user:
                if self.user_based:
                    bias = self.mu[u]
                else: 
                    bias = self.mu[i]
                a[i] = pred[i] +self.bi[i] + self.bu[u] + bias
        if len(a) < self.limit:
            recommended_items = np.argsort(a)[-self.items_count:]
        else:
            recommended_items = np.argsort(a)[-self.limit:]
        recommended_items = np.where(a[:] > 0)[0].astype(int)

#         return random.sample(list(recommended_items), self.limit)
        return recommended_items[:self.limit]
#         return recommended_items
    
    def RMSE(self, Data_test, test_size = 0, data_size = '100K'):
        n_tests = Data_test.shape[0]
        SE = 0
        for n in range(n_tests):
            pred = self.pred(Data_test[n, 0], Data_test[n, 1])
            SE += (pred - Data_test[n, 2])**2 

        RMSE = np.sqrt(SE/n_tests)
        print('RMSE =', RMSE)
        if self.user_based:
            print('%s::1::%d::%d::cosine_similarity::%r::%r\r\n' % (str(data_size), self.n_factors, self.n_epochs, test_size, RMSE))
#             self.f.write('%s::1::%d::cosine_similarity::%r::%r\r\n' % (str(data_size), self.n_factors,self.n_epochs, test_size, RMSE))
        else:
            print('%s::0::%d::%d::cosine_similarity::%r::%r\r\n' % (str(data_size), self.n_factors, self.n_epochs, test_size, RMSE))
#             self.f.write('%s::0::%d::cosine_similarity::%r::%r\r\n' % (str(data_size), self.n_factors, test_size, RMSE))
#         self.f.close()
        return RMSE
    
    def evaluate(self, data_size, Data_test, test_size = 0):
        sum_p = 0
        sum_r = 0
        self.Pu = np.zeros((self.users_count,))
        for u in range(self.users_count):
            recommended_items = self.recommend(u)
            ids = np.where(Data_test[:, 0] == u)[0]
            rated_items = Data_test[ids, 1]
            for i in recommended_items:
                if i in rated_items:
                    self.Pu[u] += 1
                if Data_test[i, 2] >= 4.5:
                    sum_r += 1
            sum_p += self.Pu[u]
        print('sump', sum_p, sum_r)
        p = sum_p/(self.users_count * self.limit)
        r = sum_p/sum_r
        print('%s::0::%d::%d::cosine_similarity::%r::%r::%r\r\n' % (str(data_size), self.n_factors, self.n_epochs, test_size, p, r))
        return p, r

In [20]:
import pandas as pd 
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from scipy import sparse 
from sklearn.model_selection import train_test_split
import random

class MF2(object):
    def __init__(self, Y, n_factors = 2, X = None, W = None, lamda = 0.1, lr = 2, n_epochs = 50, user_based = 1, 
                 limit = 10):
#         self.f = open('danhgiaMF.dat', 'a+')
        self.Y = Y
        self.lamda = lamda
        self.n_factors = n_factors
        self.lr = lr
        self.n_epochs = n_epochs
        self.user_based = user_based
        self.limit = limit
        self.users_count = int(np.max(self.Y[:, 0])) + 1
        self.items_count = int(np.max(self.Y[:, 1])) + 1
        self.ratings_count = Y.shape[0]
        if X == None:
            self.X = np.random.randn(self.items_count, n_factors)
        if W == None:
            self.W = np.random.randn(n_factors, self.users_count)
        self.Ybar = self.Y.copy()
        
        self.bi = np.random.randn(self.items_count)
        self.bu = np.random.randn(self.users_count)
        self.n_ratings = self.Y.shape[0]
        
    def normalizeY(self):
        if self.user_based:
            user_col = 0
            item_col = 1
            n_objects = self.users_count
        else:
            user_col = 1
            item_col = 0 
            n_objects = self.items_count
        users = self.Y[:, user_col]
        self.mu = np.zeros((n_objects,))
        for i in range(n_objects):
            ids = np.where(users == i)[0].astype(int)
            ratings = self.Y[ids, 2]
            m = np.mean(ratings)
            if np.isnan(m):
                m = 0
#             self.mu[i] = m
            self.Ybar[ids, 2] = ratings - self.mu[i]
#         print(self.Ybar)
    
    def get_user_rated_item(self, i):
        ids = np.where(i == self.Ybar[:, 1])[0].astype(int)
        users = self.Ybar[ids, 0].astype(int)
        ratings = self.Ybar[ids, 2]
        
        return (users, ratings)
        

    def get_item_rated_by_user(self, u):
        ids = np.where(u == self.Ybar[:, 0])[0].astype(int)
        items = self.Ybar[ids, 1].astype(int)
        ratings = self.Ybar[ids, 2]
        
        return (items, ratings)
    
    def updateX(self):
        for m in range(self.items_count):
            users, ratings = self.get_user_rated_item(m)
            Wm = self.W[:, users]
            b = self.bu[users]
            sum_grad_xm = np.full(shape = (self.X[m].shape) , fill_value = 1e-8)
            sum_grad_bm = 1e-8
            for i in range(50):
                xm = self.X[m]
                error = xm.dot(Wm) + self.bi[m] + b - ratings
                grad_xm = error.dot(Wm.T)/self.n_ratings + self.lamda*xm
                grad_bm = np.sum(error)/self.n_ratings
                sum_grad_xm += grad_xm**2
                sum_grad_bm += grad_bm**2
                # gradient descent
                self.X[m] -= self.lr*grad_xm.reshape(-1)/np.sqrt(sum_grad_xm)
                self.bi[m] -= self.lr*grad_bm/np.sqrt(sum_grad_bm)
        
    def updateW(self):
        for n in range(self.users_count):
            items, ratings = self.get_item_rated_by_user(n)
            Xn = self.X[items, :]
            b = self.bi[items]
            sum_grad_wn = np.full(shape = (self.W[:, n].shape) , fill_value = 1e-8).T
            sum_grad_bn = 1e-8
            for i in range(50):
                wn = self.W[:, n]
                error = Xn.dot(wn) + self.bu[n] + b - ratings
                grad_wn = Xn.T.dot(error)/self.n_ratings + self.lamda*wn
                grad_bn = np.sum(error)/self.n_ratings
                sum_grad_wn += grad_wn**2
                sum_grad_bn += grad_bn**2
                # gradient descent
                self.W[:, n] -= self.lr*grad_wn.reshape(-1)/np.sqrt(sum_grad_wn)
                self.bu[n] -= self.lr*grad_bn/np.sqrt(sum_grad_bn)

    def fit(self, x, data_size, Data_test, test_size = 0):
        self.normalizeY()
        for i in range(self.n_epochs):
            self.updateW()
            self.updateX()
            if (i + 1) % x == 0:
#                 print(i + 1)
                self.RMSE(Data_test, test_size = 0)
#                 self.evaluate(data_size, Data_test, test_size = 0)
            
    def pred(self, u, i):
        u = int(u)
        i = int(i)
        if self.user_based:
            bias = self.mu[u]
        else: 
            bias = self.mu[i]
        pred = self.X[i, :].dot(self.W[:, u]) + self.bi[i] + self.bu[u] + bias
        
        if pred < 1:
            return 1 
        if pred > 5: 
            return 5 
        return max(0, min(5, pred))
    
    def recommend(self, u):
        ids = np.where(self.Y[:, 0] == u)[0].astype(int)
        items_rated_by_user = self.Y[ids, 1].tolist()
        a = np.zeros((self.items_count,))
        recommended_items = []
        pred = self.X.dot(self.W[:, u])
        for i in range(self.items_count):
            if i not in items_rated_by_user:
                if self.user_based:
                    bias = self.mu[u]
                else: 
                    bias = self.mu[i]
                a[i] = pred[i] +self.bi[i] + self.bu[u] + bias
        if len(a) < self.limit:
            recommended_items = np.argsort(a)[-self.items_count:]
        else:
            recommended_items = np.argsort(a)[-self.limit:]
        recommended_items = np.where(a[:] > 0)[0].astype(int)

#         return random.sample(list(recommended_items), self.limit)
        return recommended_items[:self.limit]
#         return recommended_items
    
    def RMSE(self, Data_test, test_size = 0, data_size = '100K'):
        n_tests = Data_test.shape[0]
        SE = 0
        for n in range(n_tests):
            pred = self.pred(Data_test[n, 0], Data_test[n, 1])
            SE += (pred - Data_test[n, 2])**2 

        RMSE = np.sqrt(SE/n_tests)
        print('RMSE =', RMSE)
        if self.user_based:
            print('%s::1::%d::%d::%r::%r::%r\r\n' % (str(data_size), self.n_factors, self.n_epochs, self.lamda, self.lr, RMSE))
#             self.f.write('%s::1::%d::cosine_similarity::%r::%r\r\n' % (str(data_size), self.n_factors,self.n_epochs, test_size, RMSE))
        else:
            print('%s::0::%d::%d::cosine_similarity::%r::%r\r\n' % (str(data_size), self.n_factors, self.n_epochs, test_size, RMSE))
#             self.f.write('%s::0::%d::cosine_similarity::%r::%r\r\n' % (str(data_size), self.n_factors, test_size, RMSE))
#         self.f.close()
        return RMSE
    
    def evaluate(self, data_size, Data_test, test_size = 0):
        sum_p = 0
        sum_r = 0
        self.Pu = np.zeros((self.users_count,))
        for u in range(self.users_count):
            recommended_items = self.recommend(u)
            ids = np.where(Data_test[:, 0] == u)[0]
            rated_items = Data_test[ids, 1]
            for i in recommended_items:
                if i in rated_items:
                    self.Pu[u] += 1
                if Data_test[i, 2] >= 4.5:
                    sum_r += 1
            sum_p += self.Pu[u]
        print('sump', sum_p, sum_r)
        p = sum_p/(self.users_count * self.limit)
        r = sum_p/sum_r
        print('%s::0::%d::%d::cosine_similarity::%r::%r::%r\r\n' % (str(data_size), self.n_factors, self.n_epochs, test_size, p, r))
        return p, r

In [ ]:
r_cols = ['user_id', 'movie_id', 'rating', 'timestemp']

ratings_base_1 = pd.read_csv('ml-100k/ua.base', sep='\t', names=r_cols, encoding='latin-1', engine='python')
ratings_test_1 = pd.read_csv('ml-100k/ua.test', sep='\t', names=r_cols, encoding='latin-1', engine='python')

rate_train = ratings_base_1.as_matrix()
rate_test = ratings_test_1.as_matrix()

rate_train[:, :2] -= 1
rate_test[:, :2] -= 1


In [ ]:
# for i in [0.01, 0.1, 0,5, 1, 2]:
rs = MF(rate_train, limit =10)
rs.fit(50, '100K', Data_test = rate_test)

In [ ]:
def split_fold(rate_train, number_fold):
    index_permutation = np.random.permutation(rate_train.shape[0])
    folds = np.array_split(index_permutation , number_fold)
    return folds

def cross_validation_matrix_factorization(rate_train , rate_test, list_learning_rate , list_num_feature , list_lamda):
    dict_results = dict()
    for learning_rate in list_learning_rate:
        print("learning rate", learning_rate)
        for k in list_num_feature:
            for lamda in list_lamda:
                dict_results[(learning_rate,k,lamda)]=compute_mean_error(rate_train , rate_test, learning_rate , k , lamda)
    return dict_results
                

In [ ]:
def compute_mean_error(rate_train , rate_test, learning_rate , num_feature , lamdax):
    folds  = split_fold(rate_train , 5)
    sum_error_fold = 0
    for index_fold in range(5):
        fold_test = rate_train[folds[index_fold]]
        fold_train = np.delete(rate_train , fold_test, axis=0)
        rs = MF2(Y = fold_train, n_factors = num_feature, lamda = lamdax, lr = learning_rate, n_epochs = 10)
        rs.fit(5, '100K', fold_train)
        sum_error_fold += rs.RMSE(fold_test)
    return sum_error_fold / 5

In [9]:
learning_rate , num_feature , lamda = [0.01, 0.1 ,1 ,10, 100] , [5, 10 ,20, 50, 100] , [0.01 , 0.1 , 1]
import pickle
a = cross_validation_matrix_factorization(rate_train,rate_test,learning_rate,num_feature,lamda)
pickle.dump(a, open("result_cross","wb"))


pickle.load(open("result_cross","rb"))

5
RMSE = 1.3972735293216065
100K::1::5::10::cosine_similarity::0::1.3972735293216065

sump 0.0 4059
100K::0::5::10::cosine_similarity::0::0.0::0.0

10
RMSE = 1.0789969207214063
100K::1::5::10::cosine_similarity::0::1.0789969207214063

sump 0.0 4059
100K::0::5::10::cosine_similarity::0::0.0::0.0

RMSE = 1.1039612619826398
100K::1::5::10::cosine_similarity::0::1.1039612619826398

5
RMSE = 1.355967439483456
100K::1::5::10::cosine_similarity::0::1.355967439483456

sump 0.0 5824
100K::0::5::10::cosine_similarity::0::0.0::0.0

10
RMSE = 1.0569286334920667
100K::1::5::10::cosine_similarity::0::1.0569286334920667

sump 0.0 5833
100K::0::5::10::cosine_similarity::0::0.0::0.0

RMSE = 1.0677666843066858
100K::1::5::10::cosine_similarity::0::1.0677666843066858

5
RMSE = 1.3622238312845127
100K::1::5::10::cosine_similarity::0::1.3622238312845127

sump 0.0 5747
100K::0::5::10::cosine_similarity::0::0.0::0.0

10
RMSE = 1.0741546532757167
100K::1::5::10::cosine_similarity::0::1.0741546532757167

sump 

sump 0.0 5021
100K::0::10::10::cosine_similarity::0::0.0::0.0

RMSE = 1.1120987856713602
100K::1::10::10::cosine_similarity::0::1.1120987856713602

5
RMSE = 1.4695896109677906
100K::1::10::10::cosine_similarity::0::1.4695896109677906

sump 0.0 5108
100K::0::10::10::cosine_similarity::0::0.0::0.0

10
RMSE = 1.0808019784325442
100K::1::10::10::cosine_similarity::0::1.0808019784325442

sump 0.0 5174
100K::0::10::10::cosine_similarity::0::0.0::0.0

RMSE = 1.1117310391873974
100K::1::10::10::cosine_similarity::0::1.1117310391873974

5
RMSE = 1.4607513029660293
100K::1::10::10::cosine_similarity::0::1.4607513029660293

sump 0.0 4201
100K::0::10::10::cosine_similarity::0::0.0::0.0

10
RMSE = 1.0867340627984854
100K::1::10::10::cosine_similarity::0::1.0867340627984854

sump 0.0 4191
100K::0::10::10::cosine_similarity::0::0.0::0.0

RMSE = 1.1011811296941716
100K::1::10::10::cosine_similarity::0::1.1011811296941716

5
RMSE = 1.4467249873733388
100K::1::10::10::cosine_similarity::0::1.44672498737

5
RMSE = 1.8064074537174215
100K::1::50::10::cosine_similarity::0::1.8064074537174215

sump 0.0 4482
100K::0::50::10::cosine_similarity::0::0.0::0.0

10
RMSE = 1.1853828639684671
100K::1::50::10::cosine_similarity::0::1.1853828639684671

sump 0.0 4287
100K::0::50::10::cosine_similarity::0::0.0::0.0

RMSE = 1.191972903020588
100K::1::50::10::cosine_similarity::0::1.191972903020588

5
RMSE = 1.78945555185276
100K::1::50::10::cosine_similarity::0::1.78945555185276

sump 0.0 4956
100K::0::50::10::cosine_similarity::0::0.0::0.0

10
RMSE = 1.175193190661542
100K::1::50::10::cosine_similarity::0::1.175193190661542

sump 0.0 5054
100K::0::50::10::cosine_similarity::0::0.0::0.0

RMSE = 1.1903129373976928
100K::1::50::10::cosine_similarity::0::1.1903129373976928

5
RMSE = 1.8171355493723123
100K::1::50::10::cosine_similarity::0::1.8171355493723123

sump 0.0 3291
100K::0::50::10::cosine_similarity::0::0.0::0.0

10
RMSE = 1.1816940626520507
100K::1::50::10::cosine_similarity::0::1.1816940626520507

sump 0.0 4761
100K::0::100::10::cosine_similarity::0::0.0::0.0

10
RMSE = 1.268394963565215
100K::1::100::10::cosine_similarity::0::1.268394963565215

sump 0.0 4723
100K::0::100::10::cosine_similarity::0::0.0::0.0

RMSE = 1.2724373858223863
100K::1::100::10::cosine_similarity::0::1.2724373858223863

5
RMSE = 1.9603573471336913
100K::1::100::10::cosine_similarity::0::1.9603573471336913

sump 0.0 5868
100K::0::100::10::cosine_similarity::0::0.0::0.0

10
RMSE = 1.283366057311233
100K::1::100::10::cosine_similarity::0::1.283366057311233

sump 0.0 6114
100K::0::100::10::cosine_similarity::0::0.0::0.0

RMSE = 1.2995581122480149
100K::1::100::10::cosine_similarity::0::1.2995581122480149

5
RMSE = 1.9640489989310186
100K::1::100::10::cosine_similarity::0::1.9640489989310186

sump 0.0 3904
100K::0::100::10::cosine_similarity::0::0.0::0.0

10
RMSE = 1.2914657301070702
100K::1::100::10::cosine_similarity::0::1.2914657301070702

sump 0.0 3811
100K::0::100::10::cosine_similarity::0::0.0::0.0

RMSE 

sump 0.0 4805
100K::0::10::10::cosine_similarity::0::0.0::0.0

RMSE = 0.9729542744694436
100K::1::10::10::cosine_similarity::0::0.9729542744694436

5
RMSE = 0.9490591855519412
100K::1::10::10::cosine_similarity::0::0.9490591855519412

sump 0.0 5800
100K::0::10::10::cosine_similarity::0::0.0::0.0

10
RMSE = 0.9489019389045127
100K::1::10::10::cosine_similarity::0::0.9489019389045127

sump 0.0 5800
100K::0::10::10::cosine_similarity::0::0.0::0.0

RMSE = 0.970525630079411
100K::1::10::10::cosine_similarity::0::0.970525630079411

5
RMSE = 0.9489760321444823
100K::1::10::10::cosine_similarity::0::0.9489760321444823

sump 0.0 4232
100K::0::10::10::cosine_similarity::0::0.0::0.0

10
RMSE = 0.9488678621420285
100K::1::10::10::cosine_similarity::0::0.9488678621420285

sump 0.0 4232
100K::0::10::10::cosine_similarity::0::0.0::0.0

RMSE = 0.9652691142250995
100K::1::10::10::cosine_similarity::0::0.9652691142250995

5
RMSE = 0.949080501736986
100K::1::10::10::cosine_similarity::0::0.94908050173698

...

sump 0.0 4708
100K::0::5::10::cosine_similarity::0::0.0::0.0

RMSE = 0.9683503031117221
100K::1::5::10::cosine_similarity::0::0.9683503031117221

5
RMSE = 0.9488313654730637
100K::1::5::10::cosine_similarity::0::0.9488313654730637

sump 0.0 5549
100K::0::5::10::cosine_similarity::0::0.0::0.0

10
RMSE = 0.948792279468294
100K::1::5::10::cosine_similarity::0::0.948792279468294

sump 0.0 5549
100K::0::5::10::cosine_similarity::0::0.0::0.0

RMSE = 0.985282797489639
100K::1::5::10::cosine_similarity::0::0.985282797489639

5
RMSE = 0.9488281378406602
100K::1::5::10::cosine_similarity::0::0.9488281378406602

sump 0.0 3566
100K::0::5::10::cosine_similarity::0::0.0::0.0

10
RMSE = 0.9487932054489374
100K::1::5::10::cosine_similarity::0::0.9487932054489374

sump 0.0 3567
100K::0::5::10::cosine_similarity::0::0.0::0.0

RMSE = 0.9733416185785586
100K::1::5::10::cosine_similarity::0::0.9733416185785586

5
RMSE = 0.9489234654643033
100K::1::5::10::cosine_similarity::0::0.9489234654643033

sump 0.0 5

sump 0.0 6603
100K::0::10::10::cosine_similarity::0::0.0::0.0

10
RMSE = 0.9488583144721159
100K::1::10::10::cosine_similarity::0::0.9488583144721159

sump 0.0 6603
100K::0::10::10::cosine_similarity::0::0.0::0.0

RMSE = 0.9749967744331546
100K::1::10::10::cosine_similarity::0::0.9749967744331546

5
RMSE = 0.9487967653534914
100K::1::10::10::cosine_similarity::0::0.9487967653534914

sump 0.0 3408
100K::0::10::10::cosine_similarity::0::0.0::0.0

10
RMSE = 0.9487461799441521
100K::1::10::10::cosine_similarity::0::0.9487461799441521

sump 0.0 3408
100K::0::10::10::cosine_similarity::0::0.0::0.0

RMSE = 0.9766153413519576
100K::1::10::10::cosine_similarity::0::0.9766153413519576

5
RMSE = 0.9488591797250031
100K::1::10::10::cosine_similarity::0::0.9488591797250031

sump 0.0 6985
100K::0::10::10::cosine_similarity::0::0.0::0.0

10
RMSE = 0.948810499398599
100K::1::10::10::cosine_similarity::0::0.948810499398599

sump 0.0 6985
100K::0::10::10::cosine_similarity::0::0.0::0.0

RMSE = 0.9767264

sump 0.0 5405
100K::0::50::10::cosine_similarity::0::0.0::0.0

RMSE = 0.9789882140720372
100K::1::50::10::cosine_similarity::0::0.9789882140720372

5
RMSE = 0.9490070204382764
100K::1::50::10::cosine_similarity::0::0.9490070204382764

sump 0.0 2529
100K::0::50::10::cosine_similarity::0::0.0::0.0

10
RMSE = 0.9489579986264496
100K::1::50::10::cosine_similarity::0::0.9489579986264496

sump 0.0 2529
100K::0::50::10::cosine_similarity::0::0.0::0.0

RMSE = 0.9738074616249429
100K::1::50::10::cosine_similarity::0::0.9738074616249429

5
RMSE = 0.9489857848802634
100K::1::50::10::cosine_similarity::0::0.9489857848802634

sump 0.0 4637
100K::0::50::10::cosine_similarity::0::0.0::0.0

10
RMSE = 0.9489413084212298
100K::1::50::10::cosine_similarity::0::0.9489413084212298

sump 0.0 4637
100K::0::50::10::cosine_similarity::0::0.0::0.0

RMSE = 0.9787136523843463
100K::1::50::10::cosine_similarity::0::0.9787136523843463

5
RMSE = 0.9488874053387558
100K::1::50::10::cosine_similarity::0::0.94888740533

5
RMSE = 0.9486875469643175
100K::1::100::10::cosine_similarity::0::0.9486875469643175

sump 0.0 4143
100K::0::100::10::cosine_similarity::0::0.0::0.0

10
RMSE = 0.9486530403949439
100K::1::100::10::cosine_similarity::0::0.9486530403949439

sump 0.0 4143
100K::0::100::10::cosine_similarity::0::0.0::0.0

RMSE = 0.9714445887623636
100K::1::100::10::cosine_similarity::0::0.9714445887623636

5
RMSE = 0.9488480148871802
100K::1::100::10::cosine_similarity::0::0.9488480148871802

sump 0.0 4456
100K::0::100::10::cosine_similarity::0::0.0::0.0

10
RMSE = 0.9488067041183778
100K::1::100::10::cosine_similarity::0::0.9488067041183778

sump 0.0 4456
100K::0::100::10::cosine_similarity::0::0.0::0.0

RMSE = 0.9593549226856918
100K::1::100::10::cosine_similarity::0::0.9593549226856918

5
RMSE = 0.9487721768094195
100K::1::100::10::cosine_similarity::0::0.9487721768094195

sump 0.0 3641
100K::0::100::10::cosine_similarity::0::0.0::0.0

10
RMSE = 0.9487315133374229
100K::1::100::10::cosine_similarity::

{(0.01, 5, 0.01): 1.0859005812962392,
 (0.01, 5, 0.1): 1.0900066774482355,
 (0.01, 5, 1): 1.0893355385391776,
 (0.01, 10, 0.01): 1.1002142923236196,
 (0.01, 10, 0.1): 1.1022086866014973,
 (0.01, 10, 1): 1.1060555624344348,
 (0.01, 20, 0.01): 1.1174772664195087,
 (0.01, 20, 0.1): 1.1228076756757088,
 (0.01, 20, 1): 1.125352753614589,
 (0.01, 50, 0.01): 1.1957833857021782,
 (0.01, 50, 0.1): 1.1930501522642982,
 (0.01, 50, 1): 1.1936965565172386,
 (0.01, 100, 0.01): 1.2842221752357363,
 (0.01, 100, 0.1): 1.3026633442658564,
 (0.01, 100, 1): 1.29903546585405,
 (0.1, 5, 0.01): 0.9757543807994982,
 (0.1, 5, 0.1): 0.9755930558884509,
 (0.1, 5, 1): 0.9752771089157208,
 (0.1, 10, 0.01): 0.9725156249565124,
 (0.1, 10, 0.1): 0.9709499535175551,
 (0.1, 10, 1): 0.9761972169515873,
 (0.1, 20, 0.01): 0.9747726165005147,
 (0.1, 20, 0.1): 0.9744673719231678,
 (0.1, 20, 1): 0.9732053361626258,
 (0.1, 50, 0.01): 0.9748671607294718,
 (0.1, 50, 0.1): 0.9716847831816351,
 (0.1, 50, 1): 0.975801550565007,
 (

In [14]:
list_learning_rate , list_num_feature , list_lamda = [0.01, 0.1 ,1 ,10, 100] , [5, 10 ,20, 50, 100] , [0.01 , 0.1 , 1]
for l in list_learning_rate:
        for k in list_num_feature:
            for lam in list_lamda:
                r = MF2(rate_train, n_factors= k, lamda=lam , lr=l, n_epochs=50)
                r.fit(50, '100K', rate_test)

RMSE = 0.9873582099587408
100K::1::5::50::0.01::0.01::0.9873582099587408

sump 278.0 1767
100K::0::5::50::cosine_similarity::0::0.029480381760339344::0.15732880588568196

RMSE = 0.9872016229325654
100K::1::5::50::0.1::0.01::0.9872016229325654

sump 278.0 1767
100K::0::5::50::cosine_similarity::0::0.029480381760339344::0.15732880588568196

RMSE = 0.9871741372812957
100K::1::5::50::1::0.01::0.9871741372812957

sump 278.0 1767
100K::0::5::50::cosine_similarity::0::0.029480381760339344::0.15732880588568196

RMSE = 0.9877861749210066
100K::1::10::50::0.01::0.01::0.9877861749210066

sump 278.0 1767
100K::0::10::50::cosine_similarity::0::0.029480381760339344::0.15732880588568196

RMSE = 0.9876474551925855
100K::1::10::50::0.1::0.01::0.9876474551925855

sump 278.0 1767
100K::0::10::50::cosine_similarity::0::0.029480381760339344::0.15732880588568196

RMSE = 0.9878090692871617
100K::1::10::50::1::0.01::0.9878090692871617

sump 278.0 1767
100K::0::10::50::cosine_similarity::0::0.02948038176033934

sump 278.0 1767
100K::0::10::50::cosine_similarity::0::0.029480381760339344::0.15732880588568196

RMSE = 0.9874009018681837
100K::1::10::50::0.1::10::0.9874009018681837

sump 278.0 1767
100K::0::10::50::cosine_similarity::0::0.029480381760339344::0.15732880588568196

RMSE = 0.9871745012163954
100K::1::10::50::1::10::0.9871745012163954

sump 278.0 1767
100K::0::10::50::cosine_similarity::0::0.029480381760339344::0.15732880588568196

RMSE = 0.9871324589392088
100K::1::20::50::0.01::10::0.9871324589392088

sump 278.0 1767
100K::0::20::50::cosine_similarity::0::0.029480381760339344::0.15732880588568196

RMSE = 0.987182631042833
100K::1::20::50::0.1::10::0.987182631042833

sump 278.0 1767
100K::0::20::50::cosine_similarity::0::0.029480381760339344::0.15732880588568196

RMSE = 0.9877650148933205
100K::1::20::50::1::10::0.9877650148933205

sump 278.0 1767
100K::0::20::50::cosine_similarity::0::0.029480381760339344::0.15732880588568196

RMSE = 0.9873931839434558
100K::1::50::50::0.01::10::0.98

In [21]:
for i in [0.01, 0.1, 0.3, 0.5, 1]:
    for j in [0.1, 0.3, 0.5, 0.75, 1, 2]:
        r2 = MF2(rate_train, n_factors= 2, lamda=i , lr=j, n_epochs=50)
        r2.fit(50, '100K', rate_test)

RMSE = 0.963045194594062
100K::1::2::50::0.01::0.1::0.963045194594062

RMSE = 0.9627613610715805
100K::1::2::50::0.01::0.3::0.9627613610715805



KeyboardInterrupt: 

In [ ]:
100K::1::2::50::0.5::2::0.9629483818255299
                                                
RMSE = 0.9872454455864409
100K::1::2::50::0.01::0.1::0.9872454455864409

sump 278.0 1767
100K::0::2::50::cosine_similarity::0::0.029480381760339344::0.15732880588568196

RMSE = 0.9873509671283598
100K::1::2::50::0.01::0.3::0.9873509671283598

sump 278.0 1767
100K::0::2::50::cosine_similarity::0::0.029480381760339344::0.15732880588568196

RMSE = 0.9873022541026244
100K::1::2::50::0.01::0.5::0.9873022541026244

sump 278.0 1767
100K::0::2::50::cosine_similarity::0::0.029480381760339344::0.15732880588568196

RMSE = 0.9877267988361391
100K::1::2::50::0.01::0.75::0.9877267988361391

sump 278.0 1767
100K::0::2::50::cosine_similarity::0::0.029480381760339344::0.15732880588568196

RMSE = 0.98713629602769
100K::1::2::50::0.01::1::0.98713629602769

sump 278.0 1767
100K::0::2::50::cosine_similarity::0::0.029480381760339344::0.15732880588568196

RMSE = 0.9875245079386287
100K::1::2::50::0.01::2::0.9875245079386287

sump 278.0 1767
100K::0::2::50::cosine_similarity::0::0.029480381760339344::0.15732880588568196

RMSE = 0.9872283917511645
100K::1::2::50::0.5::0.1::0.9872283917511645

sump 278.0 1767
100K::0::2::50::cosine_similarity::0::0.029480381760339344::0.15732880588568196

RMSE = 0.9872762384675742
100K::1::2::50::0.5::0.3::0.9872762384675742

sump 278.0 1767
100K::0::2::50::cosine_similarity::0::0.029480381760339344::0.15732880588568196

RMSE = 0.9877360132473315
100K::1::2::50::0.5::0.5::0.9877360132473315

sump 278.0 1767
100K::0::2::50::cosine_similarity::0::0.029480381760339344::0.15732880588568196

RMSE = 0.9872972844633078
100K::1::2::50::0.5::0.75::0.9872972844633078

sump 278.0 1767
100K::0::2::50::cosine_similarity::0::0.029480381760339344::0.15732880588568196

RMSE = 0.9875384760756831
100K::1::2::50::0.5::1::0.9875384760756831

sump 278.0 1767
100K::0::2::50::cosine_similarity::0::0.029480381760339344::0.15732880588568196

RMSE = 0.9870764569905993
100K::1::2::50::0.5::2::0.9870764569905993

sump 278.0 1767
100K::0::2::50::cosine_similarity::0::0.029480381760339344::0.15732880588568196

RMSE = 0.9871152977603546
100K::1::2::50::1::0.1::0.9871152977603546

sump 278.0 1767
100K::0::2::50::cosine_similarity::0::0.029480381760339344::0.15732880588568196

RMSE = 0.9873422736671628
100K::1::2::50::1::0.3::0.9873422736671628

sump 278.0 1767
100K::0::2::50::cosine_similarity::0::0.029480381760339344::0.15732880588568196

RMSE = 0.9872401168654538
100K::1::2::50::1::0.5::0.9872401168654538

sump 278.0 1767
100K::0::2::50::cosine_similarity::0::0.029480381760339344::0.15732880588568196

RMSE = 0.9871853910291641
100K::1::2::50::1::0.75::0.9871853910291641

sump 278.0 1767
100K::0::2::50::cosine_similarity::0::0.029480381760339344::0.15732880588568196

RMSE = 0.9874076226831203
100K::1::2::50::1::1::0.9874076226831203

sump 278.0 1767
100K::0::2::50::cosine_similarity::0::0.029480381760339344::0.15732880588568196

RMSE = 0.9873559923225996
100K::1::2::50::1::2::0.9873559923225996

sump 278.0 1767
100K::0::2::50::cosine_similarity::0::0.029480381760339344::0.15732880588568196